In [1]:
import pandas as pd

In [18]:
data = pd.read_csv('fpp_final_table.csv')

In [19]:
data.count()

koi                     7280
koi.1                   7280
pr_heb                  5930
pr_eb                   5930
pr_beb                  5930
fp                      5977
FPP                     5930
rp                      7002
disposition             7280
prob_ontarget           3721
not_transitlike         7280
signficant_secondary    7280
centroid_offset         7280
ephem_match             7280
secdepth                6040
r_excl                  6040
dtype: int64

In [15]:
confirmed = data['disposition'] == 'CONFIRMED'
known_fp = data['disposition'] == 'FALSE POSITIVE'
candidate = data['disposition'] == 'CANDIDATE'
low_fpp = data['FPP'] < 0.01
high_fpp = data['FPP'] > 0.9

In [16]:
(low_fpp & candidate).sum()

1896

In [17]:
(high_fpp & candidate).sum()

260